In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.0.8'

# Utilisation de word embeddings


n autre moyen populaire et puissant d'associer un vecteur à un mot est l'utilisation de "vecteurs de mots" denses, également appelés "word embeddings".


Alors que les vecteurs obtenus par le codage one-hot sont binaires, clairsemés (principalement des zéros) et de très haute dimension (même dimension que le nombre de mots du vocabulaire), les "imbrications de mots" sont des vecteurs à virgule flottante de petite dimension (c'est-à-dire des vecteurs "denses", par opposition aux vecteurs clairsemés).


Contrairement aux vecteurs de mots obtenus via un codage one-hot, les intégrations de mots sont apprises à partir de données.


Il est courant de voir des combinaisons de mots de 256, 512 ou 1024 dimensions s’agissant de vocabulaires très volumineux.

![word embeddings vs. one hot encoding](https://s3.amazonaws.com/book.keras.io/img/ch6/word_embeddings.png)

Il existe deux façons d'obtenir des word embeddings :

* Apprenez les techniques d’incorporation de mots conjointement à la tâche principale qui vous tient à cœur (par exemple, la classification de documents ou la prédiction de sentiments).

Dans cette configuration, vous commencerez par des vecteurs de mots aléatoires, puis apprendrez vos vecteurs de mots de la même manière que vous apprendrez les poids d'un réseau de neurones.


* Chargez dans votre modèle des mots incorporés qui ont été pré-calculés à l'aide d'une tâche d'apprentissage machine différente de celle que vous tentez de résoudre.

Celles-ci sont appelées "embedings de mots pré-formés".

## Apprentissage avec la couche `Embedding` de Keras


In [3]:
from keras.layers import Embedding

# The Embedding layer takes at least two arguments:
# the number of possible tokens, here 1000 (1 + maximum word index),
# and the dimensionality of the embeddings, here 64.
embedding_layer = Embedding(1000, 64)


La couche `Embedding` se comprend mieux comme un dictionnaire mappant des indices entiers (qui représentent des mots spécifiques) en vecteurs denses. 

Il prend en tant qu'entrées des entiers, il recherche ces entiers dans un dictionnaire interne et renvoie les vecteurs associés.

In [1]:
from keras.datasets import imdb
from keras import preprocessing

# Number of words to consider as features
max_features = 10000
# Cut texts after this number of words 
# (among top max_features most common words)
maxlen = 20

# Load the data as lists of integers.
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# This turns our lists of integers
# into a 2D integer tensor of shape `(samples, maxlen)`
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

C:\Users\s4d-asus-14\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras.models import Sequential
from keras.layers import Flatten, Dense,Embedding

model = Sequential()
# We specify the maximum input length to our Embedding layer
# so we can later flatten the embedded inputs
model.add(Embedding(10000, 8, input_length=maxlen))
# After the Embedding layer, 
# our activations have shape `(samples, maxlen, 8)`.

# We flatten the 3D tensor of embeddings 
# into a 2D tensor of shape `(samples, maxlen * 8)`
model.add(Flatten())

# We add the classifier on top
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 3s 170us/step - loss: 0.6759 - acc: 0.6042 - val_loss: 0.6398 - val_acc: 0.6810
Epoch 2/10
20000/20000 [==============================] - 2s 106us/step - loss: 0.5657 - acc: 0.7428 - val_loss: 0.5467 - val_acc: 0.7206
Epoch 3/10
20000/20000 [==============================] - 2s 105us/step - los